# Reliability analysis of a system

In this example, we will demonstrate how to perform a reliability analysis of a system consisting of two components. We will analyze two system configurations:
* Parallel System: The system fails only when both components fail.
* Series System: The system fails when at least one of the components fails.

### Define Limit State Functions

First, we import the necessary package:

In [1]:
from ptk import *

We consider two elements, each described by the following limit state functions:

$Z_1 = 1.9 - (a+b)$

$Z_2 = 1.85 - (1.5 \cdot b + 0.5 \cdot c)$

Note that both functions share a common variable $b$.

We consider two system configurations:
* Parallel system: the system fails only when both $Z_1$ and $Z_2$ are less than zero.
* Series system: the system fails when either $Z_1$ or $Z_2$ is less than zero.

In the first case we want to calculate the probability $P(Z_1<0 \cap Z_2<0)$ and, in the second case, the probability $P(Z_1<0 \cup Z_2<0)$.

In [2]:
def limit_state_fuction_1(a, b):
    return 1.9 - (a+b)

def limit_state_fuction_2(b, c):
    return 1.85 - (1.5 * b + 0.5 * c)

### Perform reliability calculations for each element

First, we perform the reliability analysis of the individual elements (described by the limit state functions). 

We create a reliability project `ReliabilityProject()`. We assume that variables $a$ and $b$ are uniformly distributed over the interval $[-1, 1]$. The variable $c$ is normally distributed with a `mean` of $0.1$ and a `deviation` of $0.8$. We use the First Order Reliability Method (`FORM`) to run the reliability analysis.

It is essential to define the limit state function model before specifying the random variables!

Using `project.run()` we execute the reliability analysis for one limit state function. The results are stored in `project.design_point`. To combine the results later, we store the outputs in separate objects (`dp_Z1` and `dp_Z2`).

In [3]:
project = ReliabilityProject()
project.settings.reliability_method = "form"
project.settings.variation_coefficient = 0.02
project.settings.maximum_iterations = 50

project.model = limit_state_fuction_1

project.variables["a"].distribution = "uniform"
project.variables["a"].minimum = -1
project.variables["a"].maximum = 1

project.variables["b"].distribution = "uniform"
project.variables["b"].minimum = -1
project.variables["b"].maximum = 1

project.run()
dp_Z1 = project.design_point

project.model = limit_state_fuction_2

project.variables["c"].distribution = "normal"
project.variables["c"].mean = 0.1
project.variables["c"].deviation = 0.8

project.run()
dp_Z2 = project.design_point

print(f"reliability index Z1 = {dp_Z1.reliability_index}")
print(f"reliability index Z2 = {dp_Z2.reliability_index}")

reliability index Z1 = 2.772351852475011
reliability index Z2 = 1.9500014680701803


### Perform reliability analysis of a system

To perform the reliability analysis of a system, we create a new project using `CombineProject()`. We append the reliability results of the individual elements to this object.

In [4]:
combine_project = CombineProject()

combine_project.design_points.append(dp_Z1)
combine_project.design_points.append(dp_Z2)

The library offers the following methods for the reliability analysis of a system: `hohenbichler`, `importance_sampling`, `directional_sampling`. The desired reliability method can be set using `combine_project.settings.combiner_method`.

The type of system configuration (`series` or `parallel`) can be defined using `combine_project.combine_type`.

Execute the reliability analysis of the system using `combine_project.run()`. The results will be stored in `combine_project.design_point`.

In [5]:
combine_algorithms = ["hohenbichler", "importance_sampling", "directional_sampling"]

def fault_tree(combine_project, combine_type):

    print(f"{combine_type} system:")
    combine_project.settings.combine_type = combine_type

    for combine_algorithm in combine_algorithms:

        combine_project.settings.combiner_method = combine_algorithm
        combine_project.run()
        dp = combine_project.design_point

        print(f"{combine_algorithm}: reliability index = {dp.reliability_index}")

fault_tree(combine_project, "series")
fault_tree(combine_project, "parallel")

series system:
hohenbichler: reliability index = 1.9195396093665054
importance_sampling: reliability index = 1.9466454552628505
directional_sampling: reliability index = 2.028760425473061
parallel system:
hohenbichler: reliability index = 3.267257048645706
importance_sampling: reliability index = 3.127104168746118
directional_sampling: reliability index = 3.1330823165717643


### Uncorrelated $b$ variable

In the previous calculations, the variable $b$ in the limit state function $Z_1$​ was assumed to be the same as the variable $b$ in the limit state function $Z_2$​, implying a perfect correlation (correlation coefficient of $1.0$). This section demonstrates the impact of this assumption on the reliability analysis results.

To explore the effect of relaxing this assumption, we will now assume that the variable $b$ in $Z_1$ is not correlated with the variable $b$ in $Z_2$. Specifically, we set the correlation coefficient to $0.0$ using `combine_project.correlation_matrix["b"]`.

In [6]:
combine_project.correlation_matrix["b"] = 0.0

fault_tree(combine_project, "series")
fault_tree(combine_project, "parallel")


series system:
hohenbichler: reliability index = 1.9063961778047613
importance_sampling: reliability index = 1.90906385176863
directional_sampling: reliability index = 1.9404242060916543
parallel system:
hohenbichler: reliability index = 3.80395450482239
importance_sampling: reliability index = 4.308016087862686
directional_sampling: reliability index = 4.253531909948378
